In [1]:
from __future__ import print_function

import datetime
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC, SVC
from sklearn import tree

In [2]:
gongshang = pd.read_csv('price_gongshang.csv')
gongshang.columns[:-2]


Index(['date', 'open', 'high', 'close', 'low', 'volume', 'price_change',
       'p_change', 'ma5', 'ma10', 'ma20', 'v_ma5', 'v_ma10', 'v_ma20',
       'turnover', 'mom_label', 'moment', 'cv', 'volatility', 'pres1', 'pres2',
       'npres1'],
      dtype='object')

In [3]:
lags=2
gongshanglag = pd.DataFrame(index=gongshang.index)
gongshanglag["close"] = gongshang["close"]
gongshanglag["open"] = gongshang["open"]
gongshanglag["high"] = gongshang["high"]
gongshanglag["low"] = gongshang["low"]
gongshanglag["moment"] = gongshang["moment"]
gongshanglag["volatility"] = gongshang["volatility"]
gongshanglag["npres1"] = gongshang["npres1"]
gongshanglag["npres2"] = gongshang["npres2"]


gongshanglag["volume"] = gongshang["volume"]
for i in range(0, lags):
    gongshanglag["Lag%s" % str(i+1)] = gongshang["close"].shift(i+1)
    
for i in range(0, 1):
    gongshanglag["open%s" % str(i+1)] = gongshang["open"].shift(i+1)
    gongshanglag["close%s" % str(i+1)] = gongshang["close"].shift(i+1)
    gongshanglag["high%s" % str(i+1)] = gongshang["high"].shift(i+1)
    gongshanglag["low%s" % str(i+1)] = gongshang["low"].shift(i+1)
    gongshanglag["moment%s" % str(i+1)] = gongshang["moment"].shift(i+1)
    gongshanglag["volatility%s" % str(i+1)] = gongshang["volatility"].shift(i+1)
    gongshanglag["npres1%s" % str(i+1)] = gongshang["npres1"].shift(i+1)
    gongshanglag["npres2%s" % str(i+1)] = gongshang["npres2"].shift(i+1)

In [11]:
    # Create the returns DataFrame
gsret = pd.DataFrame(index=gongshanglag.index)
gsret["volume"] = gongshanglag["volume"]
gsret["close"] = gongshanglag["close"].pct_change()*100.0
gsret["open"]=gongshanglag["open"] 
#gsret["close"]=gongshanglag["close"] 

gsret["high"]=gongshanglag["high"] 
gsret["low"]=gongshanglag["low"]
gsret["moment"]=gongshanglag["moment"] 
gsret["volatility"]=gongshanglag["volatility"] 
gsret["npres1"]=gongshanglag["npres1"] 
gsret["npres2"]=gongshanglag["npres2"] 

In [12]:
for i,x in enumerate(gsret["close"]):
    if (abs(x) < 0.0001):
        gsret["close"][i] = 0.0001
gsret["close"][0] = gsret["close"][1]

In [13]:
for i in range(0, lags):
    gsret["Lag%s" % str(i+1)] = \
    gongshanglag["Lag%s" % str(i+1)].pct_change()*100.0
    
for i in range(0, 1):
    gsret["open%s" % str(i+1)] = \
    gongshanglag["open%s" % str(i+1)]
    gsret["close%s" % str(i+1)] = \
    gongshanglag["close%s" % str(i+1)]
    gsret["high%s" % str(i+1)] = \
    gongshanglag["high%s" % str(i+1)]
    gsret["low%s" % str(i+1)] = \
    gongshanglag["low%s" % str(i+1)]
    gsret["moment%s" % str(i+1)] = \
    gongshanglag["moment%s" % str(i+1)]
    gsret["volatility%s" % str(i+1)] = \
    gongshanglag["volatility%s" % str(i+1)]
    gsret["npres1%s" % str(i+1)] = \
    gongshanglag["npres1%s" % str(i+1)]
    gsret["npres2%s" % str(i+1)] = \
    gongshanglag["npres2%s" % str(i+1)]

In [86]:
gsret["label"] = np.sign(gsret["close"])
gsret['Lag1'][:2] = 0.0001
gsret['Lag2'][:3] = 0.0001
gsret['open1'][0] = gsret['open'][0]
gsret['close1'][0] = gongshanglag['close'][0]

gsret['high1'][0] = gsret['high'][0]
gsret['low1'][0] = gsret['low'][0]
gsret['moment1'][0] = gsret['moment'][0]
gsret['volatility1'][0] = gsret['volatility'][0]
gsret['npres11'][0] = gsret['npres1'][0]
gsret['npres21'][0] = gsret['npres2'][0]

scaler = MinMaxScaler()
gsret_scaled = pd.DataFrame(scaler.fit_transform(gsret[['close','open1','close1','high1','low1','moment1', 'volatility1','npres11', 'npres21', 'Lag1','Lag2']]), columns=gsret[['close','open1','close1','high1','low1','moment1', 'volatility1','npres11', 'npres21','Lag1','Lag2']].columns)
gsret_scaled["label"] = np.sign(gsret["close"])
gsret_scaled.columns


Index(['close', 'open1', 'close1', 'high1', 'low1', 'moment1', 'volatility1',
       'npres11', 'npres21', 'Lag1', 'Lag2', 'label'],
      dtype='object')

In [294]:
X = gsret_scaled[["open1","high1", 'low1','close1']]
X1 = gsret_scaled[["Lag1","Lag2"]]
X2 = gsret_scaled[["open1","high1", 'low1','close1','volatility1']]
X3 = gsret_scaled[['npres11','npres21']]

y = gsret_scaled["label"]

In [295]:
start_test = int(0.7*len(X.index))
# Create training and test sets
X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]

X_train1 = X1[X1.index < start_test]
X_test1 = X1[X1.index >= start_test]

X_train2 = X2[X2.index < start_test]
X_test2 = X2[X2.index >= start_test]

X_train3 = X3[X3.index < start_test]
X_test3 = X3[X3.index >= start_test]

y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

In [372]:
models = [("LR", LogisticRegression()), 
              ("LSVC", LinearSVC()),
              #("RSVM", SVC(
               # C=1000000.0, cache_size=200, class_weight=None,
               # coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
               # max_iter=-1, probability=False, random_state=None,
               # shrinking=True, tol=0.001, verbose=False)
              #),
              ("DT", tree.DecisionTreeClassifier(max_depth=5)),
              ("RF", RandomForestClassifier(
                n_estimators=100, criterion='gini', 
                max_depth=4, min_samples_split=3, 
                min_samples_leaf=1, max_features='auto', 
                bootstrap=True, oob_score=False, n_jobs=1, 
                random_state=None, verbose=0)),
               ("GBT", GradientBoostingClassifier(learning_rate=0.01,
                n_estimators=100, subsample=0.9, min_samples_split=3, max_depth=2))
              ]

In [373]:
for m in models:
    # Train each of the models on the training set
    m[1].fit(X_train, y_train)
    
    # Make an array of predictions on the test set
    pred = m[1].predict(X_test)
    print("%s:\n%0.3f" % (m[0], m[1].score(X_test, y_test)))
    #print("%s\n" % confusion_matrix(pred, y_test))

LR:
0.450
LSVC:
0.441
DT:
0.486
RF:
0.477
GBT:
0.514


In [606]:
back_test = gongshang[gongshang.index >= start_test]
money = 1000
n = 100
share = 0
track = []
for i in range(1,len(pred)):
    price = back_test['close'][start_test+i-1]
    track.append(money + share*price)
    if track[-1] < 0.5 * 1000:
        money = track[-1]
        share = 0
        continue
    ret=np.zeros(len(pred))
    ret[0] = 0
    ret[i] = back_test['close'][start_test+i] - back_test['close'][start_test+i-1]
    if pred[i] == 1:
        if money > n * price:
            share += n
            money -= n *price
        
    else: 
        share -= n
        money += n * price


In [608]:
back_test = gongshang[gongshang.index >= start_test]
money = 1000
n = 100
share = 0
track1 = []
for i in range(1,len(pred)):
    price = back_test['close'][start_test+i-1]
    track1.append(money + share*price)
    if track1[-1] < 0.5 * 1000:
        money = track1[-1]
        share = 0
        continue
    ret=np.zeros(len(pred))
    ret[0] = 0
    ret[i] = back_test['close'][start_test+i] - back_test['close'][start_test+i-1]
    if pred1[i] == 1:
        if money > n * price:
            share += n
            money -= n *price
        
    else: 
        share -= n
        money += n * price


In [603]:
back_test = gongshang[gongshang.index >= start_test]
money = 1000
n = 100
share = 0
track2 = []
for i in range(1,len(pred)):
    price = back_test['close'][start_test+i-1]
    track2.append(money + share*price)
    if track2[-1] < 0.5 * 1000:
        money = track2[-1]
        share = 0
        continue
    ret=np.zeros(len(pred))
    ret[0] = 0
    ret[i] = back_test['close'][start_test+i] - back_test['close'][start_test+i-1]
    if pred2[i] == 1:
        if money > n * price:
            share += n
            money -= n *price
        
    else: 
        share -= n
        money += n * price


In [621]:
back_test = gongshang[gongshang.index >= start_test]
money = 1000
n = 100
share = 0
track3 = []
for i in range(1,len(pred)):
    price = back_test['close'][start_test+i-1]
    track3.append(money + share*price)
    if track3[-1] < 0.5 * 1000:
        money = track3[-1]
        share = 0
        continue
    ret=np.zeros(len(pred))
    ret[0] = 0
    ret[i] = back_test['close'][start_test+i] - back_test['close'][start_test+i-1]
    if pred3[i] == 1:
        if money > n * price:
            share += n
            money -= n *price
        
    else: 
        share -= n
        money += n * price

In [624]:
def get_the_para(track, bechmark):
    r= np.exp(np.log(track[-1]/track[0])/(len(track)/252)) - 1
    r_b = np.exp(np.log(bechmark[-1]/bechmark[0])/(len(bechmark)/252)) - 1
    sharp = []
    bench = []
    r_f = 0.05
    r_f_d = np.exp(np.log(1 + r_f)/252) - 1
    
    for i in range(1,len(track)):
        sharp.append(track[i]/track[i-1]-1 -r_f_d)
    sharp = np.sqrt(252) * (np.array(sharp).mean())/(np.array(sharp).std())
    max_draw = 0
    max_l = 0
    min_l = max(track)
    for i in range(len(track)):
        if track[i] > max_l:
            max_l = track[i]
        if track[i] < min_l:
            min_l = track[i]
            max_draw = max(max_draw, (max_l - min_l)/max_l)
    return {'Annual Return:':r, "BenchMark:": r_b, "Annual Sharpe Ratio:":sharp, "max_drawdown:":max_draw}

In [625]:
price = back_test['close']
price = list(price)
get_the_para(track, price)

{'Annual Return:': -0.94057852562989752,
 'Annual Sharpe Ratio:': -1.560560758954751,
 'BenchMark:': 0.42429886718074528,
 'max_drawdown:': 0.93180015860428234}

In [626]:
price = back_test['close']
price = list(price)
get_the_para(track1, price)

{'Annual Return:': 0.45829508837952893,
 'Annual Sharpe Ratio:': 1.7941209187465936,
 'BenchMark:': 0.42429886718074528,
 'max_drawdown:': 0.015999999999999886}

In [627]:
price = back_test['close']
price = list(price)
get_the_para(track2, price)

{'Annual Return:': -0.61932299819363079,
 'Annual Sharpe Ratio:': -1.3028293634473143,
 'BenchMark:': 0.42429886718074528,
 'max_drawdown:': 0.68786127167630062}

In [628]:
price = back_test['close']
price = list(price)
get_the_para(track3, price)

{'Annual Return:': 0.53349981737823149,
 'Annual Sharpe Ratio:': 1.9449207307496319,
 'BenchMark:': 0.42429886718074528,
 'max_drawdown:': 0.019920318725099376}

In [615]:
def get_the_para(track, bechmark):
    r= np.exp(np.log(track[-1]/track[0])/(len(track)/252)) - 1
    r_b = np.exp(np.log(bechmark[-1]/bechmark[0])/(len(bechmark)/252)) - 1
    sharp = ((np.array(track).mean() -1000)/1000 - 0.05)/(np.array(track).std() * np.sqrt(len(track)/252))
    max_draw = 0
    max_l = 0
    min_l = max(track)
    for i in range(len(track)):
        if track[i] > max_l:
            max_l = track[i]
        if track[i] < min_l:
            min_l = track[i]
            max_draw = max(max_draw, (max_l - min_l)/max_l)
    return {'Annual Return:':r, "BenchMark:": r_b, "Annual Sharpe Ratio:":sharp, "max_drawdown:":max_draw}

#get_the_para(track, price)

In [422]:
models1 = [("LR", LogisticRegression()), 
              
              ("SVM", SVC(
                C=1000000.0, cache_size=200, class_weight=None,
                coef0=0.0, degree=3, gamma=0.01, kernel='poly',
                max_iter=-1, probability=False, random_state=None,
                shrinking=True, tol=0.001, verbose=False)
              ),
              ("DT", tree.DecisionTreeClassifier(max_depth=8)),
              ("RF", RandomForestClassifier(
                n_estimators=100, criterion='gini', 
                max_depth=4, min_samples_split=3, 
                min_samples_leaf=1, max_features='auto', 
                bootstrap=True, oob_score=False, n_jobs=1, 
                random_state=None, verbose=0)),
               ("GBT", GradientBoostingClassifier(learning_rate=0.01,
                n_estimators=100, subsample=0.9, min_samples_split=3, max_depth=4))
              ]

In [423]:
for m in models1:
    # Train each of the models on the training set
    m[1].fit(X_train1, y_train)
    
    # Make an array of predictions on the test set
    pred1 = m[1].predict(X_test1)

    # Output the hit-rate and the confusion matrix for each model
    print("%s:\n%0.3f" % (m[0], m[1].score(X_test1, y_test)))

LR:
0.573
SVM:
0.573
DT:
0.564
RF:
0.577
GBT:
0.582


In [470]:
models2 = [("LR", LogisticRegression()), 
              
              ("SVM", SVC(
                C=1000000.0, cache_size=200, class_weight=None,
                coef0=0.0, degree=3, gamma=0.01, kernel='poly',
                max_iter=-1, probability=False, random_state=None,
                shrinking=True, tol=0.001, verbose=False)
              ),
              ("DT", tree.DecisionTreeClassifier(max_depth=5)),
              ("RF", RandomForestClassifier(
                n_estimators=100, criterion='gini', 
                max_depth=2, min_samples_split=3, 
                min_samples_leaf=1, max_features='auto', 
                bootstrap=True, oob_score=False, n_jobs=1, 
                random_state=None, verbose=0)),
               ("GBT", GradientBoostingClassifier(learning_rate=0.01,
                n_estimators=100, subsample=0.7, min_samples_split=3, max_depth=5))
              ]

In [471]:
for m in models2:
    # Train each of the models on the training set
    m[1].fit(X_train2, y_train)
    
    # Make an array of predictions on the test set
    pred2 = m[1].predict(X_test2)

    # Output the hit-rate and the confusion matrix for each model
    print("%s:\n%0.3f" % (m[0], m[1].score(X_test2, y_test)))

LR:
0.527
SVM:
0.564
DT:
0.514
RF:
0.523
GBT:
0.559


In [543]:
1000 / back_test['close'][513] * back_test['close'][732]

1361.746361746362

In [465]:
models3 = [("LR", LogisticRegression()), 
              
              ("SVM", SVC(
                C=1000000.0, cache_size=200, class_weight=None,
                coef0=0.0, degree=3, gamma=0.01, kernel='poly',
                max_iter=-1, probability=False, random_state=None,
                shrinking=True, tol=0.001, verbose=False)
              ),
              ("DT", tree.DecisionTreeClassifier(max_depth=3)),
              ("RF", RandomForestClassifier(
                n_estimators=100, criterion='gini', 
                max_depth=2, min_samples_split=3, 
                min_samples_leaf=1, max_features='auto', 
                bootstrap=True, oob_score=False, n_jobs=1, 
                random_state=None, verbose=0)),
               ("GBT", GradientBoostingClassifier(learning_rate=0.02,
                n_estimators=100, subsample=0.9, min_samples_split=5, max_depth=2))
              ]

In [401]:
for m in models3:
    # Train each of the models on the training set
    m[1].fit(X_train3, y_train)
    
    # Make an array of predictions on the test set
    pred3 = m[1].predict(X_test3)

    # Output the hit-rate and the confusion matrix for each model
    print("%s:\n%0.3f" % (m[0], m[1].score(X_test3, y_test)))

LR:
0.595
SVM:
0.577
DT:
0.595
RF:
0.605
GBT:
0.609
